<a href="https://colab.research.google.com/github/stefkong1982/netology.ru/blob/Master/Analitika_bolshih_dannyh/Itogovaya_rabota_Analitika_bolshih_dannyh/Kondratev_Itog_Analitika_bolshih_dannyh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Итоговая работа "Аналитика больших данных"

**Преподаватель:** Алексей Кузьмин

**В данном разделе мы выступим в роли data scientist и попытаемся построить простую модель для рекомендации фильмов пользователям.**

1. Загрузите в колаб файлы по оценкам (ratings) и фильмам (movies) и создайте на их основе pandas-датафреймы



**Сформировав общий топ фильмов в прошлой практике, мы хотим сделать шаг вперед и начать советовать пользователю те фильмы, которые могли бы быть для него наиболее интересны. Наша цель - научится предсказывать оценку фильма пользователем. Для тестирования модели найдем пользователя, который поставил больше всего оценок**

2. Средствами Pandas, используя dataframe ratings, найдите id пользователя, поставившего больше всего оценок



**Отберем фильмы, которые оценил данный пользователь**

3. Оставьте в датафрейме ratings только те фильмы, который оценил данный пользователь



**Для построения модели нам нужны признаки. В качестве таковых будем использовать:**

  * `Год выхода`

  * `Жанры`

  * `Общее количество оценок`

  * `Суммарную оценку`




4. Добавьте к датафрейму из задания 3 столбцы:

  * По жанрам. Каждый столбец - это жанр. Единицу записываем, если фильм принадлежит данному жанру и 0 - если нет

  * столбцы с общим количеством оценок от всех пользователей на фильм и суммарной оценкой от всех пользователей



**Теперь все готово и можно строить модель**


5. Сформируйте X_train, X_test, y_train, y_test

6. Возьмите модель линейной регрессии (или любую другую для задачи регрессии)  и обучите ее на фильмах

7. Оцените качество модели на X_test, y_test при помощи метрик для задачи регрессии


**Вторая часть практики Python связана со Spark’ом**
8. Загрузить данные в spark
9. Средствами спарка вывести среднюю оценку для каждого фильма
10. Посчитайте средствами спарка среднюю оценку для каждого жанра
11. В спарке получить 2 датафрейма с 5-ю самыми популярными и самыми непопулярными фильмами (по количеству оценок, либо по самой оценке - на Ваш выбор)
